# Self-Driving Car Engineer Nanodegree

## Computer Vision

## Project: Advanced Lane Finding

### Step 0: Declare and import dependencies on modules. Also declare global variables

In [28]:
#Declare dependencies on python modules here
import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
from moviepy.editor import VideoFileClip

In [62]:
#Declare global variables used here
calibration_images_path = "camera_cal/calibration*.jpg"
distorted_images_path = "distorted_images/distorted*.jpg"
test_images_path = "test_images/*.jpg"
#Since there are 6 object points along rows and 9 object points along columns
num_object_points = (9, 6)
output_folder_path = "output_images/"
#Initialize camera calibration coefficients
camera_cal_coeff = {
    "mtx": 0,
    "dist": 0
}
#Declare and define source weights to be multiplied with imagexsize and imageysize respectively while taking a transform
perspective_src_weights = np.float32([[0.1758, 0.9722], [0.4336, 0.6597], [0.5781, 0.6597], [0.8906, 0.9722]])
#perspective_src_weights = np.float32([[0.1641, 0.9722], [0.4102, 0.6597], [0.5625, 0.6597], [0.875, 0.9722]])
#Declare and define destination weights to be multiplied with imagexsize and imageysize respectively while taking a transform
perspective_dst_weights = np.float32([[0.15625, 1], [0.15625, 0.1389], [0.8203, 0.1389], [0.8203, 1]])
#Set the number of sliding windows
nb_sliding_windows = 10
# Set the width of the windows +/- margin
sliding_window_margin = 50
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension
#Cached Lane lines list
global cached_lane_line
cached_lane_line = None

### Step 1: Compute the camera calibration matrix and distortion coefficients given a set of chessboard images

#### Steps to follow:
1. Read calibration images using glob API
2. Create object points(3D array) for one image, this will be same for all other images
3. Detect image points in each calibration image
4. Add object points and image points of particular image to object points and image points array respectively
5. Calculate camera matrix and distortion coefficients

#### Utility method to get path of images along with image name

In [30]:
def get_image_paths(path):
    images = glob.glob(path)
    return images

#### Method to find chessboard corners and draw on calibration image if specified. This method also calculates camera calibration mtx and dist coefficients

In [31]:
def calculate_camera_calibration(draw_corners=False):
    images = get_image_paths(calibration_images_path)
    obj_points = []
    img_points = []
    current_obj_point = np.zeros((num_object_points[1]*num_object_points[0], 3), np.float32)
    current_obj_point[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    
    for image_name in images:
        image = cv2.imread(image_name)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(image_gray, num_object_points, None)
        
        if ret == True:
            current_img_point = corners
            obj_points.append(current_obj_point)
            img_points.append(current_img_point)
            
            if (draw_corners): 
                cv2.drawChessboardCorners(image, num_object_points, corners, ret)
                plt.imshow(image)
                plt.show()
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, image_gray.shape[::-1], None, None)
    return (mtx, dist)

### Step 2: Apply a distortion correction to raw images

#### Steps to follow:
1. Read distorted.jpg image
2. Apply distortion correction
3. Save undistorted image

#### Method to undistort (correct distortion in a distorted image)

In [32]:
def undistort(img, mtx, dist, img_file_name="distorted", write_output=False):
    undist_image = cv2.undistort(img, mtx, dist, None, mtx)
    if (write_output):
        cv2.imwrite(output_folder_path + img_file_name.split(".", -1)[0] + "-corrected." + img_file_name.split(".", -1)[1], undist_image)
    return undist_image

### Step 3: Use color transforms, gradients, etc., to create a thresholded binary image

#### Utility method to merge binary characteristics of two or more images

In [33]:
def merge_binary_images(imgArr):
    merged_image = np.zeros_like(imgArr[0])
    for image in imgArr:
        merged_image[(image == 1)] = 1
    return merged_image

#### Method to obtain gradient thresholded image with respect to specified 'x' or 'y' orient

In [34]:
def sobel_threshold(img, sobel_kernel=3, orient="x", thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if orient == "x":
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

#### Method to obtain magnitude thresholded image from sobel derivatives

In [35]:
def magnitude_threshold(img, sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scaled_gradmag = np.uint8(255*gradmag/np.max(gradmag))
    binary_output = np.zeros_like(scaled_gradmag)
    binary_output[(scaled_gradmag >= thresh[0]) & (scaled_gradmag <= thresh[1])] = 1
    return binary_output

#### Method to obtain direction thresholded image from sobel derivatives

In [36]:
def direction_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobelx_abs = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    sobely_abs = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    graddir = np.arctan2(sobely_abs, sobelx_abs)
    binary_output = np.zeros_like(graddir)
    binary_output[(graddir >= thresh[0]) & (graddir <= thresh[1])] = 1
    return binary_output

#### Method to obtain combined image from applied gradient thresholds. In this project, gradient in 'x' direction and magnitude component of overall gradient in used for detecting lane lines

In [37]:
def apply_grad_threshold(img):
    gradx = sobel_threshold(img, sobel_kernel=11, orient="x", thresh=(40, 150))
    mag = magnitude_threshold(img, sobel_kernel=11, thresh=(50, 150))
    merged_grad_image = merge_binary_images([gradx, mag])
    return merged_grad_image

#### Apply color threshold to image.

In [38]:
def apply_color_threshold(img):
    #Declare 'R' channel threshold in RGB image
    r_thresh = (220, 255)
    #Declare 'S' channel threshold in HLS image
    s_thresh = (160, 200)
    
    #Extract 'R' channel in BGR image and apply threshold
    r_img = img[:,:,2]
    r_thresholded = np.zeros_like(r_img)
    r_thresholded[(r_img >= r_thresh[0]) & (r_img <= r_thresh[1])] = 1

    #Extract 'S' channel from HLS image and apply threshold
    s_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)[:,:,2]
    s_thresholded = np.zeros_like(s_img)
    s_thresholded[(s_img >= s_thresh[0]) & (s_img <= s_thresh[1])] = 1

    #Use characteristics derived from 'R' thresholded image with 'S' thresholded image
    combined_img = np.zeros_like(s_thresholded)
    combined_img[(r_thresholded == 1) | (s_thresholded == 1)] = 1
    return combined_img

#### Method to combine color and gradient thresholds to obtain threshloded image with lane lines detected

In [39]:
def detect_lane_lines(img):
    grad_thresholded = apply_grad_threshold(img)
    color_thresholded = apply_color_threshold(img)
    
    #Use characteristics derived from combined gradient image with combined color thresholded image
    color_and_grad = np.zeros_like(color_thresholded)
    color_and_grad[(grad_thresholded == 1) | (color_thresholded == 1)] = 1
    return color_and_grad

### Step 4: Apply a perspective transform to rectify binary image ("birds-eye view")

#### Method to apply perspective transform based on selected trapezoidal points in original image

In [40]:
def apply_perspective_transform(img):
    img_size = img.shape
    
    #Calculate actual pixel points by multiplying weights with x and y dimensions of image
    src = np.concatenate(([perspective_src_weights.T[0]*img_size[1]], [perspective_src_weights.T[1]*img_size[0]]), axis=0).T
    dst = np.concatenate(([perspective_dst_weights.T[0]*img_size[1]], [perspective_dst_weights.T[1]*img_size[0]]), axis=0).T
    
    #Calculate perspective matrix
    perspective_M = cv2.getPerspectiveTransform(src, dst)
    
    #Calculate inverse perspective matrix
    perspective_Minv = cv2.getPerspectiveTransform(dst, src)
    
    #Warp image to get a bird's eye view
    warped_img = cv2.warpPerspective(img, perspective_M, (img_size[1], img_size[0]), flags=cv2.INTER_LINEAR)
    return (warped_img, perspective_M, perspective_Minv)

### Step 5: Detect lane pixels and fit to find the lane boundary

#### Sliding window search. This method is used for searching lane lines in warped image and fitting them to a polynomial. Most of the code used in this method is re-used from the one shared in 'Project: Advanced Lane Detection' class and is attributed Udacity. Few changes made to the algorithm are as follows:

    1. Histogram is taken for bottom 66% of the image and then starting point of lane line detection is derived.
    2. Minimum number of pixels found to recenter window is updated to 2000
    3. Margin of sliding window is set to 50 pixels right and left of the midpoint

In [41]:
def sliding_window_search(binary_warped_img):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped_img[binary_warped_img.shape[0]/3:,:], axis=0)
    
    # Create an output image to draw on and  visualize the result
    output_img = np.dstack((binary_warped_img, binary_warped_img, binary_warped_img))*255
    #output_img = np.copy(img)
    
    # Find the peak of the left and right halves of the histogram
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Set height of windows
    window_height = np.int(binary_warped_img.shape[0]/nb_sliding_windows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window. Initialized to left and right base
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Set minimum number of pixels found to recenter window
    minpix = 2000
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nb_sliding_windows):
        good_left_inds = []
        good_right_inds = []
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped_img.shape[0] - (window+1)*window_height
        win_y_high = binary_warped_img.shape[0] - window*window_height
        win_xleft_low = leftx_current - sliding_window_margin
        win_xleft_high = leftx_current + sliding_window_margin
        win_xright_low = rightx_current - sliding_window_margin
        win_xright_high = rightx_current + sliding_window_margin
        # Draw the windows on the visualization image
        cv2.rectangle(output_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(output_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty = np.linspace(0, binary_warped_img.shape[0]-1, binary_warped_img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    output_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    output_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    return (output_img, lefty, righty, leftx, rightx, left_fitx, right_fitx)

### Step 6: Determine the curvature of the lane and vehicle position with respect to center

#### Method to determine radius from fit polynomials for left and right lane lines

In [51]:
def extract_radius(img, lefty, righty, leftx, rightx):
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    y_eval = np.max(ploty)/2

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    if ((right_curverad < 2000 or left_curverad < 2000) and \
        abs(right_curverad - left_curverad) > 1500):
        #Measurement of one or both lane lines is higher than offset. Take average of half of the measurement
        avg_radius = (left_curverad*0.5 + right_curverad*0.5)/2
    else:
        avg_radius = (left_curverad + right_curverad)/2
    
    #Return average, left and right lane line radius
    return (avg_radius, left_curverad, right_curverad)

### Step 7: Warp the detected lane boundaries back onto the original image

#### Mehod to extract lane detected image and warp boundaries to original image

In [43]:
def warp_lanes(img, warped, perspective_Minv, left_fitx, right_fitx):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, perspective_Minv, (img.shape[1], img.shape[0]))

    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    return (result, newwarp)

#### Method to write text to the image for displaying radius of curvature

In [44]:
def write_text(img, text, text_bottom_left_corner, fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=1, color=(255, 255, 255)):
    annotated_img = np.copy(img)
    cv2.putText(annotated_img, text, text_bottom_left_corner, fontFace, fontScale, color)
    return annotated_img

### Step 8: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

#### Method to read image frames from video, apply lane finding algorithm and write lane detected image to output video

In [54]:
def laneDetectionInVideo (sourceFilePath, outputFilePath):
    originalVideoClip = VideoFileClip(sourceFilePath)
    laneDetectedClip = originalVideoClip.fl_image(advanced_lane_detection)
    %time laneDetectedClip.write_videofile(outputFilePath, audio=False)

### Process Image, Detect Lanes, Draw lane markers

#### Utility method to check for erroneous measurement

In [63]:
def check_for_erroneous_measurement(current_lane):
    global cached_lane_line

    if (cached_lane_line == None):
        cached_lane_line = current_lane
    else:
        #Perform polygon matching to check if the shape of polygon bounded by lanes is admissible
        match_ret = cv2.matchShapes(cached_lane_line.lane_polygon, current_lane.lane_polygon ,1, 0.0)
        #Dissimilarities are detected
        if (match_ret > 0.045):
            current_lane = cached_lane_line
        #If polygon matched is similar, check for avg_radius values to be in of 500m in comparison with last measurement
        else:
            if (abs(current_lane.avg_radius - cached_lane_line.avg_radius) > 1000):
                current_lane.avg_radius = cached_lane_line.avg_radius
            #There is no error in current measurement
            else:
                cached_lane_line = current_lane
                cached_lane_line.avg_radius = (cached_lane_line.avg_radius + current_lane.avg_radius)/2
    return current_lane

#### Method to run every processing step from the pipeline and draw lane markers on original image frame from video

In [47]:
def advanced_lane_detection(img, check_for_error_results=True):
    
    #Create instance of Lane class
    current_lane = Lane()
    
    #Undistort image
    distortion_corrected = undistort(img, \
                                camera_cal_coeff["mtx"], \
                                camera_cal_coeff["dist"])
    
    #Detect lane lines after applying gradient and color threshold.
    lane_detected_img = detect_lane_lines(distortion_corrected)
    
    #Apply perspective transform to get bird's eye view of lane lines
    warped_img, perspective_M, perspective_Minv = apply_perspective_transform(lane_detected_img)
    
    #Apply sliding window search algorithm to fit lane lines to a second order polynomial
    window_detected_img, lefty, righty, leftx, rightx, left_fitx, right_fitx = sliding_window_search(warped_img)
    
    #Detect lane curvature and radius
    radius, left_curverad, right_curverad = extract_radius(window_detected_img, lefty, righty, leftx, rightx)
    
    #Warp lane lines and lane detected area back to original image
    result, lane_polygon = warp_lanes(img, warped_img, perspective_Minv, left_fitx, right_fitx)
    
    #Set detected lane properties
    current_lane.set_avg_radius(radius)
    current_lane.set_lane_polygon(cv2.cvtColor(lane_polygon, cv2.COLOR_RGB2GRAY))
    
    if (check_for_error_results):
        #Check detection for erroneous measurements
        current_lane = check_for_erroneous_measurement(current_lane)
    
    #Calculate centroid of lane detected region to determine the position of vehicle w.r.t to lane lane area
    lane_polygon = cv2.cvtColor(lane_polygon, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(lane_polygon)
    cX = int(moments["m10"] / moments["m00"])
    image_x_center = img.shape[1]/2
    
    #Write radius value to image
    annotated_img = write_text(result, "Radius of curvature = " + str(current_lane.avg_radius) + "(m)", (100, 50))

    #Write vehicle position value to image
    if (cX > image_x_center):
        annotated_img = write_text(annotated_img, "Vehicle is: " + str(round((cX - image_x_center)*xm_per_pix, 4)) + "m left of center", (100, 90))
    else:
        annotated_img = write_text(annotated_img, "Vehicle is: " + str(round((image_x_center - cX)*xm_per_pix, 4)) + "m right of center", (100, 90))
    
    return annotated_img

### Caching of results to smooth the lane search

#### Declare Lane class

In [48]:
class Lane():
    def __init__(self):
        self.lane_polygon = None
        self.avg_radius = None
        
    def set_lane_polygon(self, polygon_img):
        self.lane_polygon = np.copy(polygon_img)
        
    def set_avg_radius(self, avg_radius):
        self.avg_radius = avg_radius

### Main module

In [49]:
def main(apply_on_video=False, images_path="test_images/*.jpg", images_output_path="output_images/", video_src="", video_dst=""):
    #Calculate camera calibration matrix and cache mtx and dist values
    camera_cal_coeff["mtx"], camera_cal_coeff["dist"] = calculate_camera_calibration()
    if (apply_on_video):
        laneDetectionInVideo(video_src, video_dst)
    else:
        image_paths = get_image_paths(images_path)
        for image_path in image_paths:
            image = cv2.imread(image_path)
            annotated_image = advanced_lane_detection(image, check_for_error_results=False)
            plt.imshow(annotated_image)
            plt.show()
            cv2.imwrite(images_output_path + image_path.replace("\\", "/").split("/")[-1].split(".", -1)[0] + "_lane_detected." + image_path.split(".", -1)[1], annotated_image)

#### Run lane detection pipeline on test images

In [53]:
#main(images_path=test_images_path, images_output_path=output_folder_path)

#### Run lane detection pipeline of video

In [64]:
#main(apply_on_video=True, video_src="test_video/project_video.mp4", video_dst="output_video/project_video_lane_detected.mp4")
main(apply_on_video=True, video_src="test_video/project_video.mp4", video_dst="project-solution-final.mp4")

[MoviePy] >>>> Building video project-solution-final.mp4
[MoviePy] Writing video project-solution-final.mp4


100%|█████████████████████████████████████▉| 1260/1261 [07:49<00:00,  2.82it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project-solution-final.mp4 

Wall time: 7min 52s
